# Loop Workflow Example 1

* High level approach to making a 3D model from just a bounding box (in Australia only for now). Simply run all the cells of the notebook. 
* This requires an internet connection to access remote data, for a local data version see Example 3.

## Map2Loop

In [ ]:
import time
import os
from map2loop.project import Project
from map2loop.m2l_enums import VerboseLevel
from datetime import datetime


t0 = time.time()

bbox_3d = { # region of interest for GMD paper
    "minx": 515687.31005864,
    "miny": 7473446.76593407,
    "maxx": 562666.860106543,
    "maxy": 7521273.57407786,
    "base": -3200,
    "top": 0,
}
loop_project_filename="wa_output.loop3d"

proj = Project( 
    use_australian_state_data = "WA",
    working_projection = "EPSG:28350",
    bounding_box = bbox_3d,
    verbose_level = VerboseLevel.NONE,
    loop_project_filename = loop_project_filename
)

proj.run_all()
t1 = time.time()

## Loop Structural

In [ ]:
import numpy as np
import LoopProjectFile as LPF
import LoopStructural
from LoopStructural.visualisation import LavaVuModelViewer
from LoopStructural.modelling.input.project_file import LoopProjectfileProcessor as LPFProcessor


# LoopStructural Modelling
t2 = time.time()
fault_params = {
    'interpolatortype':'FDI',
    'nelements':1e4,
}
foliation_params = {
    'interpolatortype':'FDI' , # 'interpolatortype':'PLI',
    'nelements':1e5,  # how many tetras/voxels
    'regularisation':5,
}
projFile = LPF.ProjectFile(loop_project_filename)
processedData = LPFProcessor(projFile)
processedData.foliation_properties['sg'] = foliation_params
processedData.fault_properties['interpolatortype'] = fault_params['interpolatortype']
processedData.fault_properties['nelements'] = fault_params['nelements']

model = LoopStructural.GeologicalModel.from_processor(processedData)
model.update()

# Lavavu viewer
filename = os.path.join('.','vtk','surface_name_{}.vtk')
view = LavaVuModelViewer(model)
view.nsteps=np.array([50,50,50])
for sg in model.feature_name_index:
    if( 'super' in sg):
        view.add_data(model.features[model.feature_name_index[sg]])
view.nelements = 1e5
view.add_model_surfaces(filename=filename,faults=False)
view.nelements=1e6
view.add_model_surfaces(filename=filename,strati=False,displacement_cmap = 'rainbow')
view.nsteps = np.array([200,200,200])
view.add_model()

view.lv.control.Range('alpha', label="Global Opacity")
view.lv.control.DualRange(['xmin', 'xmax'], label="x clip", step=0.01, values=[0.0,1.0])
view.lv.control.DualRange(['ymin', 'ymax'], label="y clip", step=0.01, values=[0.0,1.0])
view.lv.control.DualRange(['zmin', 'zmax'], label="z clip", step=0.01, values=[0.0,1.0])
view.lv.control.Range(command='background', range=(0,1), step=0.1, value=0.8)

# Show the control panel, including the viewer window
view.lv.control.show()
view.interactive()

t3 = time.time()

## Elapsed Time

In [ ]:
# Print element and total processing time
m2l_time = t1-t0
ls_time = t3-t2
total = t3-t0

m2l_string = f"{m2l_time} sec" if m2l_time < 60 else f"{m2l_time/60.0} min"
ls_string = f"{ls_time} sec" if ls_time < 60 else f"{ls_time/60.0} min"
total_string = f"{total} sec" if total < 60 else f"{total/60.0} min"
print(f"map2loop {m2l_string}, LoopStructural {ls_string}, Total {total_string}")

## Extract Information from project

In [ ]:
# Save raw map data to local zipped shapefiles
proj.save_mapdata_to_files(".\output_files",".shp.zip")

In [ ]:
# Draw overlay of point data on geology map (can also try 'contacts','orientations','faults')
proj.draw_geology_map(overlay="faults")

In [ ]:
# Extract estimate of the stratigraphic column
print(proj.stratigraphic_column.column)